In [2]:
import pandas as pd
import csv
import re

%matplotlib inline

In [3]:
#returns list of [market, stock_sybol] by querying database.

MAX_LIMIT = 100000
DATA_PATH = '..\\data'
US_STOCK_FILE_CSV = DATA_PATH + '\\us_stock.csv'

#limit means the number of results to be returned. 0 means everything
def getSymbols(limit=10):
    if limit <= 0:
        limit = MAX_LIMIT

    count = 0
    stocks = []

    with open(US_STOCK_FILE_CSV, 'rb') as stock_csv:
        lines = csv.reader(stock_csv, delimiter=',')
        for line in lines:
            market = line[0]
            stock = line[1]
            m = re.search('^[A-Z]+$', stock)
            if m:
                stock = m.group(0)
                stocks.append([market, stock])
                count =  count + 1
                #print count,
                if count >= limit:
                    break
    return stocks

In [4]:
#returns db column names from the dataframe arg. names are a form of string without special characters.

def getFields(df):
    res = []
    y = df.iloc[:,0:1] #extract 1st column
    #y.iloc[1:10].values
    for z in y.iloc[:].values:
        x=re.sub('[ /,\.\&\-()\']','',z[0]) #remove special chars
        if isinstance(x, unicode):
            x = x.encode('UTF-8')
            #print type(x)
        res.append(x)
    return res

In [5]:
def getTableName(data):
    y = data.iloc[:,0:1]
    val =  y.iloc[0:1].values[0][0]
    if isinstance(val, unicode):
        val = val.encode('UTF-8')
    
    if val.startswith('Revenue'):
        return 'IncomeStatement'
    elif val.startswith('Cash '):
        return 'BalanceSheet'
    elif val.startswith('Net '):
        return 'CashFlow'
    else:
        return None


In [6]:
#open files for fin values

data_to_extract = {1:'IncomeStatement', #index in the dataframe, the name of financial data
                    3:'BalanceSheet',
                    5:'CashFlow'}

#returns dic of fin data type and writer handle
def openFinFiles():
    stocks = getSymbols(1)
    stock = stocks[0]
    #print stocks
    url = "https://www.google.com/finance?q="+stock[0]+"%3A"+stock[1]+"&fstype=ii"
    #print url
    df = pd.read_html(url, encoding=False)
    #print len(df)
   
    fin_files = {}
    for k in data_to_extract.keys():
        #print k
        
        ### get header string
        x = df[k]
        y = x.iloc[:,0:]
        fields = getFields(y)
        common_fields = ['symbol','date','period']
        fields = common_fields + fields
        header = ','.join(fields)
        #print header

        file_path = DATA_PATH + '\\' + data_to_extract[k] + '.csv'
        print file_path
        
        f = open(file_path, "wb")
        f.write(header)
        f.write('\r\n')

        fin_files[data_to_extract[k]] = f
        
    return fin_files
        
def closeFinFiles(fin_files):
    for k in fin_files.keys():
        fin_files[k].close()
    return

In [7]:
#get values from fin tables for each stock

def getPeriodAndDate(column_value):
    if isinstance(column_value, unicode):
        column_value = column_value.encode('UTF-8')

    column_value = re.sub('[\n]','',column_value) #remove special chars

    size = len(column_value)
    period = column_value[:size-11]
    date = column_value[size-10:]
    return (period,date)    
    
def gatherFinValues(limit=10):
    if limit <= 0:
        limit = MAX_LIMIT
        
    fin_files = openFinFiles()
        
    stocks = getSymbols(limit) #get every symbol

    for stock in stocks[:limit]:
        market = stock[0]
        stockname = stock[1]
        url = "https://www.google.com/finance?q="+market+"%3A"+stockname+"&fstype=ii"
        print url

        df = None
        try:
            df = pd.read_html(url, encoding=False)
        except Exception:
            print "Exception has been caught while processing [%s:%s]. No financial data."%(market, stockname)
            
        if df is None:
            continue     #skip the rest if there is no financial data for this stock
            
        print 'num of data is %d'%(len(df))

        if len(df) <= 1:
            print "Exception has been caught while processing [%s:%s]. No financial data."%(market, stockname)
            continue    #skip the rest if there is no financial data for this stock
        
        #data_to_extract : this is declared already
        
        #columns_to_extract = [1,2,3,4]  #from 2015 to 2012

        for k in range(0, len(df)):
            data = df[k]
            columns_to_extract = range(1,len(data.columns.values)) # all data columns
            
            for c in columns_to_extract:
                #print getPeriodAndDate(data.columns.values[c])
                tablename = getTableName(data)
                (period, date) = getPeriodAndDate(data.columns.values[c])
                
                #common fields
                fields = [stockname, date, period]
                #row = '%s,%s,%s'%(stockname, date, period)

                #data fields
                y = data.iloc[:,c:c+1]
                for z in y.iloc[:].values:
                    try:
                        val = z[0].encode('UTF-8')
                    except AttributeError:
                        print z[0]
                        val = z[0]
                        
                    #print val
                    if val == '-':
                        fields.append('NA')
                    else:
                        fields.append(val)
                
                fields_str = ','.join(fields)
                print fields_str
                
                fin_files[tablename].write(fields_str)
                fin_files[tablename].write('\r\n')

    closeFinFiles(fin_files)
    
    #end of gatherFinValues   


In [8]:
gatherFinValues(10)

..\data\IncomeStatement.csv
..\data\BalanceSheet.csv
..\data\CashFlow.csv
https://www.google.com/finance?q=NYSE%3AA&fstype=ii
num of data is 6
A,2016-10-31,3 months ending,1111.00,NA,1111.00,523.00,588.00,321.00,84.00,NA,NA,NA,NA,928.00,183.00,NA,NA,-16.00,151.00,124.00,NA,NA,124.00,NA,NA,NA,124.00,NA,124.00,124.00,NA,NA,NA,0.60,329.00,0.38,NA,0.12,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.38
A,2016-07-31,3 months ending,1044.00,NA,1044.00,502.00,542.00,310.00,86.00,NA,NA,NA,NA,898.00,146.00,NA,NA,4.00,134.00,124.00,NA,NA,124.00,NA,NA,NA,124.00,NA,124.00,124.00,NA,NA,NA,NA,328.00,0.38,NA,0.12,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.44
A,2016-04-30,3 months ending,1019.00,NA,1019.00,489.00,530.00,318.00,81.00,NA,NA,NA,NA,888.00,131.00,NA,NA,-4.00,117.00,91.00,NA,NA,91.00,NA,NA,NA,91.00,NA,91.00,91.00,NA,NA,NA,NA,328.00,0.28,NA,0.12,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.33
A,2016-01-31,3 months ending,1028.00,NA,1028.00,491.00,537.00,304.00,78.00,NA,NA,NA,NA,873.00,155.00,NA,NA,5.00,142.00,123